In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import os
import torch
from omegaconf import OmegaConf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot  as plt
from hydra.utils import instantiate
from mofgraph2vec.data.datamodule import DataModuleFactory

In [3]:
conf_doc2label_data = OmegaConf.load("../conf/doc2label_data/rsm.yaml")
conf_doc2label_data.label_path = "../data/data.csv"
conf_doc2label_data.embedding_path = "../data/embedding_example/embedding-dv.csv"

In [11]:
conf_doc2label_data.batch_size = 128

In [4]:
dmf = DataModuleFactory(**conf_doc2label_data)

2023-03-09 12:47:14.215 | INFO     | mofgraph2vec.data.datamodule:__init__:65 - Train: 2637 Valid: 294 Test: 326


In [5]:
dm = dmf.get_datamodule()

In [6]:
from mofgraph2vec.model.vecnn import VecModel
from mofgraph2vec.model.nn_lightning import VecLightningModule

In [7]:
model_conf = OmegaConf.load("../conf/doc2label_model/deep.yaml")
trainer_conf = OmegaConf.load("../conf/trainer/default.yaml")
model_conf.nn.input_dim = 500
trainer_conf.max_epochs = 10

In [13]:
model_conf.nn.fcnn_hidden_size = 128

In [14]:
pl_model = VecLightningModule(instantiate(model_conf.nn), "mse", 1e-4)
trainer = instantiate(trainer_conf)

/Users/xiaoqi/opt/anaconda3/envs/mofgraph2vec/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/xiaoqi/opt/anaconda3/envs/mofgraph2vec/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of th

In [15]:
trainer.tune(pl_model, datamodule=dm)

/Users/xiaoqi/opt/anaconda3/envs/mofgraph2vec/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/xiaoqi/opt/anaconda3/envs/mofgraph2vec/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1600: PossibleUserWarning: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(
/Users/xiaoqi/opt/anaconda3/envs/mofgraph2vec/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
LR finder stopped early after 60 steps due to diverging loss.
Learning rate set to 0.0002089296130854041
Restoring states from the checkpoint path at /Users/xiaoqi/Documents/projects/proj_unsupervisedML/codes/mofgraph2vec/dev/.lr_find_b6c6465b-621e-4b72-9d29-a40643453d5e.ckpt
Restored all states from the checkpoint file at /Users/xiaoqi/Documents/projects/proj_unsupervisedML/codes/mofgraph2vec/dev/.lr_find_b6c6465b-621e-4b72-9d29-a40643453d5e.ckpt


{'lr_find': <pytorch_lightning.tuner.lr_finder._LRFinder at 0x7f8789b54220>}

In [16]:
trainer.fit(pl_model, datamodule=dm)


  | Name    | Type     | Params
-------------------------------------
0 | model   | VecModel | 397 K 
1 | loss    | MSELoss  | 0     
2 | dropout | Dropout  | 0     
-------------------------------------
397 K     Trainable params
0         Non-trainable params
397 K     Total params
1.589     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


## Evaluate model performance

In [ ]:
predictions = trainer.predict(loaded_model, dataloaders=dm.test_dataloader())

In [ ]:
true, pred = predictions[0][0], predictions[0][1]

In [ ]:
true = np.concatenate([pred[0].view(-1).numpy() for pred in predictions])
pred = np.concatenate([pred[1].view(-1).numpy() for pred in predictions])

In [ ]:
test_values = pd.DataFrame(np.vstack((true, pred)).T, columns=["True", "Pred"])

In [ ]:
test_values

In [ ]:
plt.scatter(true, pred) #, yerr=pred_std.reshape(-1,))

# Get embedding

In [ ]:
loaded_model = VecLightningModule.load_from_checkpoint("lightning_logs/version_6/checkpoints/epoch=49-step=2100.ckpt")

In [ ]:
pl_model.model.eval()
params = pl_model.model.parameters()

In [ ]:
pl_model.model.embedding.weight

In [26]:
df_vector = pd.read_csv("../data/embedding_example/embedding-dv.csv").set_index("type")

In [ ]:
vec = torch.Tensor(df_vector.loc["RSM0001"].values)

In [21]:
vec = torch.Tensor(df_vector.values)

In [22]:
embedd_vec = pl_model.model.get_embedding(vec).detach().numpy()

In [23]:
embedd_vec

array([[-0.01603144,  0.03246498, -0.04267263, ...,  0.02525965,
        -0.04343928,  0.03340538],
       [-0.01798633,  0.03400552, -0.04214737, ...,  0.02616085,
        -0.04202629,  0.03407082],
       [-0.01705399,  0.03398563, -0.04183646, ...,  0.02760736,
        -0.04409898,  0.03413099],
       ...,
       [-0.00518457,  0.03108034, -0.05639659, ...,  0.03510607,
        -0.02479803,  0.01728296],
       [-0.0029513 ,  0.03223477, -0.0589047 , ...,  0.03730493,
        -0.02087384,  0.01599566],
       [-0.00556529,  0.0333194 , -0.05741774, ...,  0.03920066,
        -0.02470611,  0.01877441]], dtype=float32)

In [27]:
df_vector.loc[:] = embedd_vec

In [28]:
df_vector

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,...,x_490,x_491,x_492,x_493,x_494,x_495,x_496,x_497,x_498,x_499
type,,,,,,,,,,,,,,,,,,,,,
RSM0001,-0.016031,0.032465,-0.042673,0.021358,0.023141,0.041399,-0.040029,-0.005651,0.024738,0.001733,...,0.026930,0.023326,0.038193,0.032746,0.037504,0.013662,-0.041720,0.025260,-0.043439,0.033405
RSM0002,-0.017986,0.034006,-0.042147,0.020298,0.023653,0.040903,-0.042310,-0.004479,0.024896,0.001448,...,0.027047,0.023103,0.038152,0.033030,0.037669,0.015868,-0.042211,0.026161,-0.042026,0.034071
RSM0003,-0.017054,0.033986,-0.041836,0.021678,0.023977,0.040686,-0.041274,-0.004606,0.025003,0.000299,...,0.027920,0.022153,0.037019,0.034716,0.038099,0.015971,-0.042543,0.027607,-0.044099,0.034131
RSM0004,-0.018209,0.035395,-0.041762,0.021517,0.023583,0.041144,-0.040300,-0.003707,0.024128,0.000745,...,0.026977,0.022528,0.038833,0.033908,0.037926,0.014050,-0.041886,0.026157,-0.043590,0.034471
RSM0008,-0.018226,0.032405,-0.041137,0.021723,0.024012,0.041460,-0.042055,-0.003147,0.026218,0.001976,...,0.027485,0.021573,0.038718,0.034348,0.037471,0.014976,-0.042204,0.025492,-0.044047,0.034206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RSM4610,-0.002570,0.033075,-0.057312,0.003924,0.033260,0.041043,-0.038707,-0.020604,-0.000530,-0.003665,...,0.043199,0.055154,0.056118,0.031725,0.036038,0.021569,-0.047047,0.036226,-0.023646,0.017361
RSM4613,-0.006229,0.033719,-0.057503,0.004687,0.032060,0.042331,-0.037497,-0.017881,0.004593,-0.002356,...,0.039427,0.052705,0.052647,0.032822,0.035933,0.019391,-0.047259,0.034744,-0.024674,0.019017
RSM4614,-0.005185,0.031080,-0.056397,0.003268,0.036029,0.041050,-0.037617,-0.017309,0.004579,-0.001490,...,0.042471,0.053350,0.052551,0.033067,0.037639,0.021191,-0.046958,0.035106,-0.024798,0.017283


In [29]:
from mofdscribe.datasets import QMOFDataset

In [30]:
qmof_ds = QMOFDataset()

In [34]:
qmof_ds.get_subset

TypeError: Cannot index by location index with a non-integer key

In [35]:
import json

In [37]:
with open("/Users/xiaoqi/.data/mofdscribe/QMOF/v0.0.1/data.json") as file:
    data = json.load(file)

In [43]:
df_data = pd.DataFrame.from_dict(data, orient="columns")

In [48]:
subdf = df_data[df_data['flavor.gcmc']==True]

In [46]:
data.keys()

dict_keys(['flavor.all', 'flavor.gcmc', 'flavor.csd', 'flavor.csd-gcmc', 'outputs.pbe.energy_total', 'outputs.pbe.energy_vdw', 'outputs.pbe.energy_elec', 'outputs.pbe.net_magmom', 'outputs.pbe.bandgap', 'outputs.pbe.cbm', 'outputs.pbe.vbm', 'outputs.pbe.directgap', 'outputs.pbe.bandgap_spins', 'outputs.pbe.cbm_spins', 'outputs.pbe.vbm_spins', 'outputs.pbe.directgap_spins', 'outputs.hle17.energy_total', 'outputs.hle17.energy_vdw', 'outputs.hle17.energy_elec', 'outputs.hle17.net_magmom', 'outputs.hle17.bandgap', 'outputs.hle17.cbm', 'outputs.hle17.vbm', 'outputs.hle17.directgap', 'outputs.hle17.bandgap_spins', 'outputs.hle17.cbm_spins', 'outputs.hle17.vbm_spins', 'outputs.hle17.directgap_spins', 'outputs.hse06_10hf.energy_total', 'outputs.hse06_10hf.energy_vdw', 'outputs.hse06_10hf.energy_elec', 'outputs.hse06_10hf.net_magmom', 'outputs.hse06_10hf.bandgap', 'outputs.hse06_10hf.cbm', 'outputs.hse06_10hf.vbm', 'outputs.hse06_10hf.directgap', 'outputs.hse06_10hf.bandgap_spins', 'outputs.hse

In [51]:
sum(subdf['info.density'].isna())

0